In [1]:
using Pkg
# Pkg.activate("../../")
Pkg.activate("../../", io=devnull)
Pkg.update()

    Updating registry at `C:\Users\jakdar\.julia\registries\General.toml`
   Installed Revise ─ v3.13.1
    Updating `C:\Users\jakdar\uni\bachelorarbeit\VeryDiff\Project.toml`
  [295af30f] ↑ Revise v3.13.0 ⇒ v3.13.1
  [37e2e46d] ↓ LinearAlgebra v1.12.0 ⇒ v1.11.0
  [2f01184e] ↓ SparseArrays v1.12.0 ⇒ v1.11.0
    Updating `C:\Users\jakdar\uni\bachelorarbeit\VeryDiff\Manifest.toml`
  [ac6e5ff7] - JuliaSyntaxHighlighting v1.12.0
⌅ [aea7be01] ↓ PrecompileTools v1.3.3 ⇒ v1.2.1
  [295af30f] ↑ Revise v3.13.0 ⇒ v3.13.1
  [ec057cc2] ↑ StructUtils v2.6.0 ⇒ v2.6.1
  [458c3c95] ~ OpenSSL_jll v3.5.4+0 ⇒ v3.5.4+0
  [f43a241f] ↓ Downloads v1.7.0 ⇒ v1.6.0
  [37e2e46d] ↓ LinearAlgebra v1.12.0 ⇒ v1.11.0
  [ca575930] ↓ NetworkOptions v1.3.0 ⇒ v1.2.0
  [44cfe95a] ↓ Pkg v1.12.1 ⇒ v1.11.0
  [2f01184e] ↓ SparseArrays v1.12.0 ⇒ v1.11.0
  [e66e0078] ↓ CompilerSupportLibraries_jll v1.3.0+1 ⇒ v1.1.1+0
  [781609d7] ↓ GMP_jll v6.3.0+2 ⇒ v6.3.0+0
  [deac9b47] ↓ LibCURL_jll v8.15.0+0 ⇒ v8.6.0+0
  [e37daf67] ↓ LibGit2

In [2]:
using VeryDiff
using LinearAlgebra
using Plots

include("../util.jl")

Set parameter Username
Academic license - for non-commercial use only - expires 2026-11-30


RecipesBase.apply_recipe

In [3]:
struct Result
    net :: String
    spec :: String
    status :: VeryDiff.VerificationStatus
    runtime :: Int64
    num_propagation :: Int64
    num_input_splits :: Int64
    num_neuron_splits :: Int64
    initial_δ_bound :: Float64
    final_δ_bound :: Float64
end

In [4]:
function read_results(csv_dir::String)
    results = Result[]
    open(csv_dir, "r") do f
        while !eof(f)
            line = split(readline(f), ",")
            net = replace(line[1], " " => "")
            spec = replace(line[2], " " => "")
            status = replace(line[3], " " => "")
            status = status == "SAFE" ? VeryDiff.SAFE : (status == "UNSAFE" ? VeryDiff.UNSAFE : VeryDiff.UNKNOWN)
            runtime = parse(Int64, string(line[4]))
            num_propagation = parse(Int64, string(line[5]))
            num_input_splits = parse(Int64, string(line[6]))
            num_neuron_splits = parse(Int64, string(line[7]))
            initial_δ_bound = parse(Float64, string(line[8]))
            final_δ_bound = parse(Float64, string(line[9]))
            push!(results, Result(net, spec, status, runtime, num_propagation, num_input_splits, num_neuron_splits, initial_δ_bound, final_δ_bound))
        end
    end
    return results
end

read_results (generic function with 1 method)

In [12]:
cur_dir = @__DIR__
exp_final_dir = "$cur_dir\\experiments_final"

verydiff_exp_name = "VeryDiff"
deepsplit_exp_name = "DeepSplit-Base"
deepsplit_alt_exp_name = "DeepSplit-Alt"
deepsplit_input_exp_name = "DeepSplit-Base-Input"
deepsplit_diffzono_exp_name = "DeepSplit-Base-DiffZono"
deepsplit_input_diffzono_exp_name = "DeepSplit-Base-Input-DiffZono"
deepsplit_unbiased_exp_name = "DeepSplit-Unbiased-Base"
deepsplit_unbiased_input_exp_name = "DeepSplit-Unbiased-Base-Input"
deepsplit_unbiased_diffzono_exp_name = "DeepSplit-Unbiased-Base-DiffZono"
deepsplit_unbiased_input_diffzono_exp_name = "DeepSplit-Unbiased-Base-Input-DiffZono"

acas_results = "acas-0.5\\results.csv"
mnist_results = "mnist-1.0\\results.csv"

verydiff_acas_csv_dir = joinpath(exp_final_dir, verydiff_exp_name, acas_results)
deepsplit_acas_csv_dir = joinpath(exp_final_dir, deepsplit_exp_name, acas_results)
deepsplit_alt_acas_csv_dir = joinpath(exp_final_dir, deepsplit_alt_exp_name, acas_results)
deepsplit_input_acas_csv_dir = joinpath(exp_final_dir, deepsplit_input_exp_name, acas_results)
deepsplit_diffzono_acas_csv_dir = joinpath(exp_final_dir, deepsplit_diffzono_exp_name, acas_results)
deepsplit_input_diffzono_acas_csv_dir = joinpath(exp_final_dir, deepsplit_input_diffzono_exp_name, acas_results)
deepsplit_unbiased_acas_csv_dir = joinpath(exp_final_dir, deepsplit_unbiased_exp_name, acas_results)
deepsplit_unbiased_input_acas_csv_dir = joinpath(exp_final_dir, deepsplit_unbiased_input_exp_name, acas_results)
deepsplit_unbiased_diffzono_acas_csv_dir = joinpath(exp_final_dir, deepsplit_unbiased_diffzono_exp_name, acas_results)
deepsplit_unbiased_input_diffzono_acas_csv_dir = joinpath(exp_final_dir, deepsplit_unbiased_input_diffzono_exp_name, acas_results)

verydiff_mnist_csv_dir = joinpath(exp_final_dir, verydiff_exp_name, mnist_results)
deepsplit_mnist_csv_dir = joinpath(exp_final_dir, deepsplit_exp_name, mnist_results)
deepsplit_alt_mnist_csv_dir = joinpath(exp_final_dir, deepsplit_alt_exp_name, mnist_results)
deepsplit_input_mnist_csv_dir = joinpath(exp_final_dir, deepsplit_input_exp_name, mnist_results)
deepsplit_diffzono_mnist_csv_dir = joinpath(exp_final_dir, deepsplit_diffzono_exp_name, mnist_results)
deepsplit_input_diffzono_mnist_csv_dir = joinpath(exp_final_dir, deepsplit_input_diffzono_exp_name, mnist_results)
# deepsplit_unbiased_mnist_csv_dir = joinpath(exp_final_dir, deepsplit_unbiased_exp_name, mnist_results)
# deepsplit_unbiased_input_mnist_csv_dir = joinpath(exp_final_dir, deepsplit_unbiased_input_exp_name, mnist_results)
# deepsplit_unbiased_diffzono_mnist_csv_dir = joinpath(exp_final_dir, deepsplit_unbiased_diffzono_exp_name, mnist_results)
# deepsplit_unbiased_input_diffzono_mnist_csv_dir = joinpath(exp_final_dir, deepsplit_unbiased_input_diffzono_exp_name, mnist_results)

println("...")

...


In [13]:
sort_func = res -> res.runtime

verydiff_acas_results = sort!(read_results(verydiff_acas_csv_dir), by=sort_func)
deepsplit_acas_results = sort!(read_results(deepsplit_acas_csv_dir), by=sort_func)
deepsplit_alt_acas_results = sort!(read_results(deepsplit_alt_acas_csv_dir), by=sort_func)
deepsplit_input_acas_results = sort!(read_results(deepsplit_input_acas_csv_dir), by=sort_func)
deepsplit_diffzono_acas_results = sort!(read_results(deepsplit_diffzono_acas_csv_dir), by=sort_func)
deepsplit_input_diffzono_acas_results = sort!(read_results(deepsplit_input_diffzono_acas_csv_dir), by=sort_func)
deepsplit_unbiased_acas_results = sort!(read_results(deepsplit_unbiased_acas_csv_dir), by=sort_func)
deepsplit_unbiased_input_acas_results = sort!(read_results(deepsplit_unbiased_input_acas_csv_dir), by=sort_func)
deepsplit_unbiased_diffzono_acas_results = sort!(read_results(deepsplit_unbiased_diffzono_acas_csv_dir), by=sort_func)
deepsplit_unbiased_input_diffzono_acas_results = sort!(read_results(deepsplit_unbiased_input_diffzono_acas_csv_dir), by=sort_func)

verydiff_mnist_results = sort!(read_results(verydiff_mnist_csv_dir), by=sort_func)
deepsplit_mnist_results = sort!(read_results(deepsplit_mnist_csv_dir), by=sort_func)
deepsplit_alt_mnist_results = sort!(read_results(deepsplit_alt_mnist_csv_dir), by=sort_func)
deepsplit_input_mnist_results = sort!(read_results(deepsplit_input_mnist_csv_dir), by=sort_func)
deepsplit_diffzono_mnist_results = sort!(read_results(deepsplit_diffzono_mnist_csv_dir), by=sort_func)
deepsplit_input_diffzono_mnist_results = sort!(read_results(deepsplit_input_diffzono_mnist_csv_dir), by=sort_func)
# deepsplit_unbiased_mnist_results = sort!(read_results(deepsplit_unbiased_mnist_csv_dir), by=sort_func)
# deepsplit_unbiased_input_mnist_results = sort!(read_results(deepsplit_unbiased_input_mnist_csv_dir), by=sort_func)
# deepsplit_unbiased_diffzono_mnist_results = sort!(read_results(deepsplit_unbiased_diffzono_mnist_csv_dir), by=sort_func)
# deepsplit_unbiased_input_diffzono_mnist_results = sort!(read_results(deepsplit_unbiased_input_diffzono_mnist_csv_dir), by=sort_func)

# all_acas_results = [verydiff_acas_results, deepsplit_acas_results, deepsplit_alt_acas_results, deepsplit_input_acas_results, deepsplit_diffzono_acas_results, deepsplit_input_diffzono_acas_results]
all_mnist_results = [verydiff_mnist_results, deepsplit_mnist_results, deepsplit_alt_mnist_results, deepsplit_input_mnist_results, deepsplit_diffzono_mnist_results, deepsplit_input_diffzono_mnist_results]
all_acas_results = [verydiff_acas_results, deepsplit_acas_results, deepsplit_alt_acas_results, deepsplit_input_acas_results, deepsplit_diffzono_acas_results, deepsplit_input_diffzono_acas_results, deepsplit_unbiased_acas_results, deepsplit_unbiased_input_acas_results, deepsplit_unbiased_diffzono_acas_results, deepsplit_unbiased_input_diffzono_acas_results]
# all_mnist_results = [verydiff_mnist_results, deepsplit_mnist_results, deepsplit_alt_mnist_results, deepsplit_input_mnist_results, deepsplit_diffzono_mnist_results, deepsplit_input_diffzono_mnist_results, deepsplit_unbiased_mnist_results, deepsplit_unbiased_input_mnist_results, deepsplit_unbiased_diffzono_mnist_results, deepsplit_unbiased_input_diffzono_mnist_results]

10-element Vector{Vector{Result}}:
 [Result("ACASXU_run2a_1_9_batch_2000_pruned5", "prop_4", VeryDiff.SAFE, 483285, 1, 0, 0, 0.0016847800387558521, 0.0016847800387558521), Result("ACASXU_run2a_1_9_batch_2000_pruned5", "prop_3", VeryDiff.SAFE, 513286, 1, 0, 0, 0.0006034115948338539, 0.0006034115948338539), Result("ACASXU_run2a_5_9_batch_2000_pruned5", "prop_3", VeryDiff.SAFE, 612309, 1, 0, 0, 0.4470737258867684, 0.4470737258867684), Result("ACASXU_run2a_2_9_batch_2000_pruned5", "prop_10", VeryDiff.UNSAFE, 1198747, 1, 0, 0, 13675.722857652177, 13675.722857652177), Result("ACASXU_run2a_4_1_batch_2000_pruned5", "prop_5", VeryDiff.UNSAFE, 1357577, 1, 0, 0, 3751.1877564072206, 3751.1877564072206), Result("ACASXU_run2a_2_9_batch_2000_pruned5", "prop_26", VeryDiff.UNSAFE, 1401937, 1, 0, 0, 31557.750946350257, 31557.750946350257), Result("ACASXU_run2a_4_9_batch_2000_pruned5", "prop_10", VeryDiff.UNSAFE, 1454556, 1, 0, 0, 12677.168538804644, 12677.168538804644), Result("ACASXU_run2a_2_9_batch_20

In [7]:
function count_solved(results::Vector{Result})
    count(r -> r.status != VeryDiff.UNKNOWN, results)
end

count_solved (generic function with 1 method)

In [14]:
x_acas = [1:count_solved(results) for results in all_acas_results]
y_acas = [[results[k].runtime / 1e9 for k in x_acas[i]] for (i, results) in enumerate(all_acas_results)]

10-element Vector{Vector{Float64}}:
 [0.000483285, 0.000513286, 0.000612309, 0.001198747, 0.001357577, 0.001401937, 0.001454556, 0.001456381, 0.001491398, 0.001529389  …  5.465051756, 7.579443666, 7.594409729, 8.264870775, 8.530710178, 9.05766453, 9.261310511, 9.435128518, 9.988624942, 13.862977662]
 [0.00049026, 0.000507911, 0.000758987, 0.001083083, 0.001140711, 0.001263347, 0.001342369, 0.001437841, 0.001450221, 0.001456764  …  0.002600382, 0.013181515, 0.015258433, 0.117770397, 0.157245569, 0.174232404, 0.906424509, 1.975996233, 4.56518703, 116.885871935]
 [0.000415317, 0.0004369, 0.000452795, 0.0011064, 0.001109351, 0.001169736, 0.001296943, 0.001327802, 0.001354314, 0.001462954  …  0.001659251, 0.002093318, 0.002177528, 0.130100339, 0.15741398, 0.16523416, 0.922911619, 1.987540257, 4.581955606, 116.463796039]
 [0.000391467, 0.000455443, 0.000501573, 0.001053097, 0.0011658, 0.001260351, 0.001394274, 0.001409246, 0.001461428, 0.00147348  …  8.481328218, 8.489300923, 9.043449796, 12

In [17]:
# linewidth = 2.0
plot(x_acas[1], y_acas[1], label="VeryDiff", dpi=400, title="ACAS Benchmarks", xlabel="Number of Instances Verified", ylabel="Time (s)", legend=:legend, framestyle=:origin)
plot!(x_acas[2], y_acas[2], label="DeepSplit-Base")
plot!(x_acas[3], y_acas[3], label="DeepSplit-Alt")
plot!(x_acas[4], y_acas[4], label="DeepSplit-Base-Input")
plot!(x_acas[5], y_acas[5], label="DeepSplit-Base-DiffZono")
plot!(x_acas[6], y_acas[6], label="DeepSplit-Base-Input-DiffZono")
# plot!(x_acas[7], y_acas[7], label="DeepSplit-Unbiased-Base")
# plot!(x_acas[8], y_acas[8], label="DeepSplit-Unbiased-Base-Input")
# plot!(x_acas[9], y_acas[9], label="DeepSplit-Unbiased-Base-DiffZono")
# plot!(x_acas[10], y_acas[10], label="DeepSplit-Unbiased-Base-Input-DiffZono")

savefig("acas_benchmarks.png")

"c:\\Users\\jakdar\\uni\\bachelorarbeit\\VeryDiff\\dev\\experiments\\acas_benchmarks.png"

In [10]:
x_mnist = [1:count_solved(results) for results in all_mnist_results]
y_mnist = [[results[k].runtime / 1e9 for k in x_mnist[i]] for (i, results) in enumerate(all_mnist_results)]

6-element Vector{Vector{Float64}}:
 [0.001378444, 0.001390721, 0.001406744, 0.001435391, 0.0014486, 0.001464628, 0.004250638, 0.008880225, 0.019297298, 0.025707291  …  0.632306755, 1.343554582, 1.675129774, 2.90971231, 3.509331593, 4.14777903, 4.310045306, 7.243081671, 10.514891249, 14.519371999]
 [0.001321383, 0.001337689, 0.001337743, 0.001388654, 0.001395223, 0.001743182, 0.01096307, 0.015994952, 0.017196876, 0.017225974  …  1.111158915, 1.303816637, 8.315101902, 10.200323858, 11.529918421, 25.255122155, 38.196841067, 42.256306243, 58.053966498, 62.214939992]
 [0.00130822, 0.001328432, 0.00134522, 0.001356126, 0.001373644, 0.00145005, 0.010913297, 0.01611097, 0.017161093, 0.017161108  …  1.132084046, 1.305980554, 7.814088894, 10.231877362, 11.59341381, 25.125867309, 38.121941123, 42.059183983, 57.518048118, 61.813550388]
 [0.001231715, 0.001290847, 0.001346112, 0.001347237, 0.001367323, 0.001437351, 0.003889933, 0.009515902, 0.017115266, 0.017269599  …  4.488264784, 5.106038079, 8.4

In [11]:
plot(x_mnist[1], y_mnist[1], label="VeryDiff", dpi=400, title="MNIST Benchmarks", xlabel="Number of Instances Verified", ylabel="Time (s)", legend=:legend, framestyle=:origin)
plot!(x_mnist[2], y_mnist[2], label="DeepSplit-Base")
plot!(x_mnist[3], y_mnist[3], label="DeepSplit-Alt")
plot!(x_mnist[4], y_mnist[4], label="DeepSplit-Base-Input")
plot!(x_mnist[5], y_mnist[5], label="DeepSplit-Base-DiffZono")
plot!(x_mnist[6], y_mnist[6], label="DeepSplit-Base-Input-DiffZono")
# plot!(x_mnist[7], y_mnist[7], label="DeepSplit-Unbiased-Base")
# plot!(x_mnist[8], y_mnist[8], label="DeepSplit-Unbiased-Base-Input")
# plot!(x_mnist[9], y_mnist[9], label="DeepSplit-Unbiased-Base-DiffZono")
# plot!(x_mnist[10], y_mnist[10], label="DeepSplit-Unbiased-Base-Input-DiffZono")

savefig("mnist_benchmarks.png")

"c:\\Users\\jakdar\\uni\\bachelorarbeit\\VeryDiff\\dev\\experiments\\mnist_benchmarks.png"